In [1]:
# Import libraries
from ElexonDataPortal import api
import warnings
import pandas as pd
from sp2ts import sp2ts, ts2sp, from_unixtime, sp2dt
from datetime import datetime, timedelta

In [2]:
# Initialize client to make API requests to elexon
client = api.Client("fq5wqyjmzynb1uu")

In [3]:
# Initialize start and end dates
start_date = pd.Timestamp(datetime(2022,1,1))
end_date = datetime(2022,1,2)

In [4]:
# Initialize empty list to append load dataframes
load_list = []

In [8]:
while start_date < end_date:
    # Requesting data from elexon 15 days at a time to handle API constraints
    new_end = start_date + timedelta(days=15)
    if new_end > end_date:
        new_end = end_date
#     Requesting data from day and day ahead demand https://www.bmreports.com/bmrs/?q=demand/dayanddayaheaddemand
    df = client.get_FORDAYDEM(start_date.date(), new_end.date())
    df_ = pd.DataFrame()
    df_ = df[df['recordType'] == 'DATF']
    df_.loc[:,'settlementDate'] = pd.to_datetime(df_['settlementDate']).dt.date
    df_.loc[:,'settlementPeriod'] = df_['settlementPeriod'].astype(int)
#     Converting settlement date and settlement period to datetime objects
    df_.loc[:,'datetime'] = df_.apply(lambda x: sp2dt(x['settlementDate'], x['settlementPeriod'], closed="left"), axis=1)
#     Capturing NDF, TSDF, DAID, DAIG. Refer https://www.bmreports.com/bmrs/?q=demand/dayanddayaheaddemand
    df_.loc[:,'Demand Forecast (MW)'] = df[df['recordType'] == 'DANF']['demand'].values
    df_.loc[:,'Demand Forecast (MW)'] = df_['Demand Forecast (MW)'].astype(float)
    df_.loc[:, 'Actual Total Load (MW)'] = df[df['recordType'] == 'DAID']['spnDemand'].values
    df_.loc[:,'Actual Total Load (MW)'] = df_['Actual Total Load (MW)'].astype(float).abs()
#     Aggregating data by 1 hour and calculating mean for all the features
    df_ = df_.groupby([pd.Grouper(freq='1H', key='datetime')]).agg({'Demand Forecast (MW)':'mean', 'Actual Total Load (MW)':'mean'}).reset_index()
#     Appending 15 days or less of data into the load list
    load_list.append(df_)
    start_date = new_end

/var/folders/8j/_b25ntsx2rs5l3_6fp4p3xc00000gn/T/ipykernel_28466/2284050528.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.loc[:,'settlementDate'] = pd.to_datetime(df_['settlementDate']).dt.date
/var/folders/8j/_b25ntsx2rs5l3_6fp4p3xc00000gn/T/ipykernel_28466/2284050528.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.loc[:,'settlementPeriod'] = df_['settlementPeriod'].astype(int)
/var/folders/8j/_b25ntsx2rs5l3_6fp4p3xc00000gn/T/ipykernel_28466/2284050528.py:13: SettingWithCopyWarning: 
A 

In [9]:
# Converting load_list to a dataframe
df_load = pd.concat(load_list)

In [10]:
df_load.head()

,datetime,Demand Forecast (MW),Actual Total Load (MW)
0,2022-01-01 00:00:00+00:00,21760.0,21482.0
1,2022-01-01 01:00:00+00:00,20787.0,21623.0
2,2022-01-01 02:00:00+00:00,18960.5,22955.5
3,2022-01-01 03:00:00+00:00,17343.5,22924.0
4,2022-01-01 04:00:00+00:00,16407.5,22886.0


In [11]:
# Writing outputs of forecast loads to a csv file
df_load.to_csv('forecast_load.csv', index=False)